# Kriging 


In [3]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from matplotlib.colorbar import Colorbar
from matplotlib import ticker
import rasterio
from pykrige.ok import OrdinaryKriging
import geopandas as gpd
from shapely.geometry import box
from PIL import Image
import warnings
import matplotlib.cm as cm
pd.set_option('display.max_rows', None)
warnings.filterwarnings("ignore")

In [5]:
general_path = 'C:/Users/abner/OneDrive/Documentos/mestrado/DISSERTAÇÃO/scripts'
plt.rcParams.update({'font.family': 'Times New Roman'})
# Definindo os parâmetros
years = list(range(2014, 2025))
letra = list("abcdefghijklmnopqrstuvwxyz")
mapas =['EI_30', 'rain', 'E_D']
unity = ['Erosivity (MJ.mm/ha.h.year)', 'Rainfall (mm/year)', 'Erosivity density(MJ/ha.h)']
cmap_list = ['coolwarm_r', 'RdYlBu', 'viridis_r']
color_list=['#008B8B','#D35400','#4B4B4B']


# Ler o shapefile do Brasil
brasil_uf = gpd.read_file(r'C:\Users\abner\OneDrive\Documentos\mestrado\Telemetria-ANA\Shapefile\Brasil_UF\BR_UF_2022.shp')
brasil_shapefile = brasil_uf.to_crs('EPSG:4674')

In [7]:
def custom_formatter(x, pos):
    if x >= 1e6:
        return f'{int(x*1e-6)} M'
    elif x >= 1e3:
        return f'{int(x*1e-3)} k'
    else:
        return f'{x:.1f}'

In [ ]:
#REMOVE OUTLIERS
hdf_path = os.path.join(general_path, "Results erosivity", "Erosivity_final.h5")
with pd.HDFStore(hdf_path, 'r') as store:
    keys = store.keys()
df_list = []
for key in keys:
    if key.endswith('/annual'):
        parts = key.strip('/').split('/')
        region_year = parts[0]  # ex: AC_2019
        if '_' in region_year:
            region, year = region_year.split('_')
            df = pd.read_hdf(hdf_path, key=key)
            df['region'] = region
            df['year'] = year
            df_list.append(df)

df_full = pd.concat(df_list, ignore_index=True)
df_full['year'] = df_full['year'].astype(int)
df_full['EI30'] = pd.to_numeric(df_full['EI30'], errors='coerce')
df_full.dropna(subset=['EI30'], inplace=True)

regions = sorted(df_full['region'].unique())
years = sorted(df_full['year'].unique())
output_hdf_path = os.path.join(general_path, "Results erosivity", "outliers_by_year.h5")
with pd.HDFStore(output_hdf_path, mode='w') as store:
    for year in years:
        outliers_year_list = []
        for region in regions:
            data = df_full[(df_full['year'] == year) & (df_full['region'] == region)]
            if data.empty:
                continue
            q1 = data['EI30'].quantile(0.25)
            q3 = data['EI30'].quantile(0.75)
            iqr = q3 - q1
            lower_bound = q1 - 1.5 * iqr
            upper_bound = q3 + 1.5 * iqr

            outliers = data[(data['EI30'] < lower_bound) | (data['EI30'] > upper_bound)].copy()
            if not outliers.empty:
                outliers_year_list.append(outliers[['gauge_code', 'EI30', 'region']])

        if outliers_year_list:
            outliers_df = pd.concat(outliers_year_list, ignore_index=True)
            store.put(str(year), outliers_df)
            print(f"[✓] Saved outliers for year {year} with {len(outliers_df)} stations.")
        else:
            print(f"[ ] No outliers found for year {year}.")

In [ ]:
#TABEL CLASSIFICATION
df = pd.read_hdf(f"{general_path}/Results erosivity/EI30_all_months.h5", key='annual')
years = [str(year) for year in range(2014, 2025)]
df_years = df[years]
values = df_years.values.flatten()

ranges = [
    (0.1, 2452, "Low Erosivity"),
    (2452, 4905, "Moderate Erosivity"),
    (4905, 7357, "Moderate-High Erosivity"),
    (7357, 9810, "High Erosivity"),
    (9810, None, "Very High Erosivity")
]

values = values[values != 0]
total_values = len(values)

table = []
for r in ranges:
    lower, upper, description = r
    if lower is None:
        mask = values <= upper
    elif upper is None:
        mask = values > lower
    else:
        mask = (values > lower) & (values <= upper)
    count = mask.sum()
    percent = (count / total_values) * 100 if total_values > 0 else 0
    table.append({
        "Erosivity Range (R)": f"{lower if lower is not None else ''} < R ≤ {upper if upper is not None else ''}",
        "Erosivity Level": description,
        "Observed Percentage (%)": f"{percent:.1f}%",
        "Number of Stations (2014–2024)": count
    })

df_table = pd.DataFrame(table)
output_path = os.path.join(general_path, 'Results erosivity', '1 - Result_images', 'erosivity_classification_observed.xlsx')
df_table.to_excel(output_path, index=False)
df_table

In [77]:
df_data=pd.read_hdf(f"{general_path}/Results erosivity/EI30_all_months.h5", key=key).reset_index()[['state', 'gauge_code', 'long', 'lat', str(year)]].sort_values(by='2014',ascending=False)

In [81]:
df_data[~df_data['gauge_code'].isin(lista_outliers)].sort_values(by='2014',ascending=False).head(150)

,state,gauge_code,long,lat,2014
2076,GO,24700000,-52.227800,-15.891400,4823.879084
3826,SC,83892990,-49.330000,-27.330000,4099.939228
2761,AM,11500000,-67.935600,-3.101700,3933.432889
917,RO,15564000,-62.892500,-10.831700,3814.755099
1282,AC,12590000,-70.745600,-8.151900,3534.160484
586,MT,66260001,-56.108600,-15.615600,3111.737334
263,MT,66070004,-57.702200,-16.076100,2977.150997
113,MT,17093010,-58.353600,-11.356400,2971.667100
903,AM,14330000,-66.802200,-0.200600,2534.764928
2038,PI,34933000,-42.407200,-4.393100,2410.522826


In [89]:
#Individual Maps
with pd.HDFStore(f"{general_path}/Results erosivity/EI30_all_months.h5") as store:
    keys=store.keys()
plt.rcParams.update({'font.family': 'Times New Roman'})
output_dir = os.path.join(general_path,'Results erosivity' ,"1 - Result_images")
os.makedirs(output_dir, exist_ok=True)
mapa='EI_30'
k=2

cmap = cmap_list[k]
for year in years[6:]:  
    for key in keys[1:]:
        #filter quality gauges
        df_anual = pd.read_hdf(f"{general_path}/Results erosivity/EI30_all_months.h5", key='annual').reset_index()
        valid_gauges=df_anual[['state', 'gauge_code', 'long', 'lat', str(year)]].replace(0, np.nan).dropna().reset_index(drop=True)['gauge_code'].tolist()
        df_data = pd.read_hdf(f"{general_path}/Results erosivity/EI30_all_months.h5", key=key).reset_index()[['state', 'gauge_code', 'long', 'lat', str(year)]]
        df_data = df_data[df_data['gauge_code'].isin(valid_gauges)].reset_index(drop=True)
        df_data = df_data.rename(columns={str(year): mapa})
        df_outliers = pd.read_hdf(os.path.join(general_path, "Results erosivity", "outliers_by_year.h5"), key=str(year))
        lista_outliers = df_outliers['gauge_code'].to_list()
        df_validation=pd.read_excel(os.path.join(general_path, "Results erosivity", "validation_dataset.xlsx"))
        lista_validation=df_validation['gauge_code'].to_list()
        df_data = df_data[~df_data['gauge_code'].isin(lista_outliers)]
        #df_data = df_data[~df_data['gauge_code'].isin(lista_validation)]
        
        if key == '/annual':
            df_data=df_data[df_data[mapa] < 40000]### exclude problematic value
        else:
            df_data=df_data[df_data[mapa] < 4000]### exclude problematic value
        latitude = df_data['lat'].values
        longitude = df_data['long'].values
        aqi_value =  df_data[mapa].values 
        minx, miny, maxx, maxy = brasil_shapefile.total_bounds
        extent = [minx, maxx, miny, maxy]
        gridx = np.linspace(minx, maxx, 372)
        gridy = np.linspace(miny, maxy, 372)
        OK = OrdinaryKriging(
            longitude, latitude, aqi_value,
            variogram_model='spherical',
            coordinates_type='geographic',
            verbose=False,
            nlags=5 #nlags
        )
        z_interp, ss = OK.execute('grid', gridx, gridy, backend='loop',n_closest_points=30) #, n_closest_points=200
    
        variogram_parameters = OK.variogram_model_parameters
        C0 = variogram_parameters[0]  # Nugget
        C1 = variogram_parameters[2]  # Sill - Nugget
        SDD = (C0 / (C0 + C1)) * 100
    
        fig, ax_map = plt.subplots(figsize=(10, 10))
    
        cax = ax_map.imshow(
            z_interp,
            extent=extent,
            origin='lower',
            cmap=cmap,
            alpha=1,
        )
        brasil_shapefile.plot(ax=ax_map, facecolor="none", edgecolor="black", linewidth=1)
        ax_map.scatter(
            longitude, latitude,
            color="gray", label="Gauges",
            zorder=3, marker='o', s=14, edgecolor="black", linewidth=0.45,
        )
        #ax_map.set_title(f"Ordinary Kriging  {unity[k]} - {year}: {key}", fontsize=16)
        ax_map.grid(alpha=0.6)
        ax_map.legend(fontsize=13)
        ax_map.tick_params(labelsize=16)
    
        cbar = fig.colorbar(cax, ax=ax_map, orientation='vertical', fraction=0.036, shrink=0.4, pad=0.02)
        cbar.set_label(f'{unity[k]}', fontsize=15)
        cbar.ax.tick_params(labelsize=13)
        inset_ax = fig.add_axes([0.195, 0.215, 0.21, 0.2])
        lags = OK.lags
        semivariance = OK.semivariance
        variogram_model = OK.variogram_model
        variogram_parameters = OK.variogram_model_parameters
        model_values = OK.variogram_function(variogram_parameters, lags)
    
        inset_ax.plot(lags, semivariance, 'o', label='Semivariance', alpha=0.6)
        inset_ax.plot(lags, model_values, '-', label=f'Modelo {variogram_model}', alpha=1 ,color = color_list[k])
        inset_ax.set_title("Semivariogram", fontsize=15)
        inset_ax.set_xlabel("Distance (km)", fontsize=12)
        inset_ax.set_ylabel("Semivariance", fontsize=12)
        inset_ax.legend(fontsize=12, loc='upper left', frameon=False)
        inset_ax.tick_params(labelsize=11)
        inset_ax.yaxis.set_major_formatter(ticker.FuncFormatter(custom_formatter))
        inset_ax.grid(alpha=0.7)
        inset_ax.set_facecolor((1, 1, 1, 0.4))
        sdd_text = f"SDD = {SDD:.2f}%"
        ax_map.text(0.8, 0.03, sdd_text, fontsize=16, color='black', transform=ax_map.transAxes,
                    bbox=dict(facecolor='white', alpha=0.7, boxstyle='round'))
        
        # SAVE MAP WITH SEMIVARIOGRAM
        output_dir = os.path.join(general_path,'Results erosivity' ,f'2 - Result_images_{mapa}', str(year))
        os.makedirs(output_dir, exist_ok=True)
        safe_key = key.strip('/').replace('/', '_')
        output_path_map = os.path.join(output_dir, f"Ordinary Kriging_{mapa}_{year}{key.replace('/', '_')}.png")
        plt.savefig(output_path_map, dpi=300, bbox_inches='tight', pil_kwargs={"compression": "tiff_lzw"})
    
        raster_path = os.path.join(output_dir, f"{mapa}_map_{year}{key.replace('/', '_')}.tif")
        with open(raster_path, 'wb') as f:
            Image.fromarray(z_interp).save(f)
        
        #variogram
        fig_variogram, ax_variogram = plt.subplots(figsize=(6, 6))
        ax_variogram.plot(lags, semivariance, 'o', label='Semivariance', alpha=0.7)
        ax_variogram.plot(lags, model_values, '-', label=f'Modelo {variogram_model}', alpha=1, color = color_list[k])
        ax_variogram.set_xlabel("Lag distance (km)", fontsize=20)
        ax_variogram.set_ylabel("Semivariance", fontsize=20)
        ax_variogram.legend(fontsize=16, loc='upper left', frameon=False)
        ax_variogram.tick_params(labelsize=16)
        ax_variogram.yaxis.set_major_formatter(ticker.FuncFormatter(custom_formatter))
        ax_variogram.grid(alpha=1)
        ax_variogram.text(0.6, 0.05, sdd_text, fontsize=18, color='black', transform=ax_variogram.transAxes,
                          bbox=dict(facecolor='white', alpha=0.7, boxstyle='round'))
        output_path_variogram = os.path.join(output_dir, f"Variogram_{year}_{mapa}{key.replace('/', '_')}.png")
        fig_variogram.savefig(output_path_variogram, dpi=300, bbox_inches='tight', pil_kwargs={"compression": "tiff_lzw"})
        plt.close(fig_variogram)
        plt.close(fig)
    


In [85]:
key

'/month_07'

In [87]:
year

2019

In [ ]:
#variogram collection
# Configurações do diretório de entrada e anos
input_dir = os.path.join(general_path, '2 - Result_images')
  # Anos de 2014 a 2023
years = list(range(2014, 2025))
# Criar o subplot
fig, axes = plt.subplots(4, 3, figsize=(12, 16))  
axes = axes.flatten()  # Transformar em uma lista para iterar facilmente

for i, year in enumerate(years):
    ax = axes[i]
    file_path = os.path.join(input_dir, str(year), f"Variograma_{year}_{mapa}.png")
    
    if os.path.exists(file_path):
        img = Image.open(file_path)
        ax.imshow(img)
        ax.set_title(f"{letra[i]})                  {year}", fontsize=16,loc='left')
        ax.axis('off')  # Esconder os eixos
    else:
        ax.set_title(f"{year} (Faltando)", fontsize=12, color='red')
        ax.axis('off')
for ax in axes[len(years):]:
    fig.delaxes(ax)
# Ajustar o layout
fig.suptitle(f"Comparação de Variogramas (2014-2024) {mapa}", fontsize=16)
fig.tight_layout(rect=[0, 0, 1, 0.95])

# Salvar e exibir o gráfico
output_path_comparison = os.path.join(input_dir, f"Comparacao_Variogramas_{mapa}_2014_2024.png")
plt.savefig(output_path_comparison, dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
#all RASTERS ALL YEARS
plt.rcParams.update({'font.family': 'Times New Roman'})
for k, mapa in enumerate(mapas):


    
    bounds = brasil_shapefile.total_bounds
    minx, miny, maxx, maxy = bounds
    extent = [minx, maxx, miny, maxy]
    gridx = np.linspace(minx, maxx, 150)
    gridy = np.linspace(miny, maxy, 150)
    z_min, z_max = float('inf'), float('-inf')
    cmap = cmap_list[k]
    nrows, ncols = 3, 4
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(16, 12), constrained_layout=True)
    # Processamento dos anos
    for i, year in enumerate(years):

        df_data = pd.read_hdf(f"{general_path}/Results_Erosivity_zero.h5", key='EI_30')
        df_data = df_data[['state', 'gauge_code', 'long', 'lat', str(year)]].replace(0, np.nan).dropna().reset_index(drop=True)
        df_data = df_data.rename(columns={str(year): 'EI_30'})
        df_data['E_D'] = pd.read_hdf(f"{general_path}/Results_Erosivity_zero.h5", key='E_D')[[str(year)]].replace(0, np.nan).dropna().reset_index(drop=True)[str(year)]
        df_data['rain'] = df_data['EI_30'] / df_data['E_D']
    
        df_outliers = pd.read_hdf(os.path.join(general_path, '2 - Result_images', 'outliers_removidos.h5'), key=str(year))
        lista_outliers = df_outliers['gauge_code'].to_list()
        df_data = df_data[~df_data['gauge_code'].isin(lista_outliers)]
    
        latitude = df_data['lat'].values
        longitude = df_data['long'].values
        aqi_value = df_data[mapa].values

        output_dir = os.path.join(general_path, '2 - Result_images', 'Results-dissertação',str(year))
        if mapa == 'rain':output_dir = os.path.join(general_path, '2 - Result_images',str(year))
        raster_path = os.path.join(output_dir, f"{mapa}_map_{year}.tif")
        with rasterio.open(raster_path) as f:
            z_interp = f.read(1)


        if mapa == 'EI_30':
            z_interp = np.where(z_interp > 31000, 31000, z_interp)  # Define o limite máximo
        if mapa == 'rain':
            z_interp = np.where(z_interp > 3500, 3500, z_interp)  # Define o limite máximo

        z_min = min(z_min, z_interp.min())
        z_max = max(z_max, z_interp.max())

        ax = axes[i // ncols, i % ncols]
        im = ax.imshow(z_interp, extent=extent, origin='lower', cmap=cmap, alpha=1, vmin=None, vmax=None)
        brasil_shapefile.plot(ax=ax, facecolor="none", edgecolor="black", linewidth=1)
        ax.scatter(longitude, latitude, color="k", zorder=3, marker='o', s=4)
        ax.set_title(f"{letra[i]})  {year}", fontsize=20)
        ax.set_xlim([minx, maxx])
        ax.set_ylim([miny, maxy])
        ax.grid(color='gray', linestyle='--', linewidth=0.5)
        ax.set_aspect('equal', 'box')
        for spine in ax.spines.values():
            spine.set_edgecolor('black')
            spine.set_linewidth(1)
        ax.tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)  # Remove os números das bordas
    for ax in axes.flat:
        im.set_clim(z_min, z_max)
    axes[-1, -1].axis("off")
    pos = axes[-1, -1].get_position()  # Posição do último subplot
    cbar_ax = fig.add_axes([pos.x0+0.1, pos.y0, 0.03 , pos.height-0.03])  # Usa a mesma posição e tamanho
    norm = cm.colors.Normalize(vmin=z_min, vmax=z_max)
    cbar = Colorbar(ax=cbar_ax, mappable=cm.ScalarMappable(norm=norm, cmap=cmap), orientation='vertical')
    cbar.ax.tick_params(labelsize=16)
    cbar.set_label(unity[k], fontsize=18)
    plt.savefig(general_path + r'\2 - Result_images\Krigagem_ordinária_'+mapa+'_All_years', dpi=300, bbox_inches='tight')
    plt.close()

In [ ]:
   #HISTOGRAMAS

for k, mapa in enumerate(mapas):
    for year in range(2014, 2025):
        df_data = pd.read_hdf(f"{general_path}/Results_Erosivity_zero.h5", key='EI_30')
        df_data = df_data[['state', 'gauge_code', 'long', 'lat', str(year)]].replace(0, np.nan).dropna().reset_index(drop=True)
        df_data = df_data.rename(columns={str(year): 'EI_30'})
        df_data['E_D'] = pd.read_hdf(f"{general_path}/Results_Erosivity_zero.h5", key='E_D')[[str(year)]].replace(0, np.nan).dropna().reset_index(drop=True)[str(year)]
        df_data['rain'] = df_data['EI_30'] / df_data['E_D']
    
        df_outliers = pd.read_hdf(os.path.join(general_path, '2 - Result_images', 'outliers_removidos.h5'), key=str(year))
        lista_outliers = df_outliers['gauge_code'].to_list()
        df_data = df_data[~df_data['gauge_code'].isin(lista_outliers)]
    
        latitude = df_data['lat'].values
        longitude = df_data['long'].values
        aqi_value = df_data[mapa].values
        # Filtrar valores menores que 30.000
        aqi_value = aqi_value[aqi_value < 30000]
    
        # Salvar o mapa com o variograma
        output_dir = os.path.join(general_path, '2 - Result_images', str(year))
        os.makedirs(output_dir, exist_ok=True)
        
        # Gerar e salvar o gráfico de histograma
        plt.figure(figsize=(8, 6))
        plt.hist(aqi_value, bins=20, color=color_list[k], edgecolor='black', alpha=0.7)
        plt.xlabel(f'{unity[k]}', fontsize=22)      
        plt.ylabel('Frequência', fontsize=22)
        plt.grid(True, linestyle='--', alpha=0.9)
        plt.gca().tick_params(labelsize=20)
        plt.gca().xaxis.set_major_formatter(ticker.FuncFormatter(custom_formatter))
        
        # Salvar histograma
        output_path_histogram = os.path.join(output_dir, f"Histograma_{year}_{mapa}.png")
        plt.savefig(output_path_histogram, dpi=300, bbox_inches='tight')
        plt.close()  # Fechar a figura para liberar memória

In [ ]:
#HISTOGRAM COLLECTION
# Definindo a fonte global para Times New Roman
plt.rcParams.update({'font.family': 'Times New Roman'})
input_dir = os.path.join(general_path, '2 - Result_images')

for mapa in mapas:
    fig, axes = plt.subplots(3, 4, figsize=(16, 12))  # 4 colunas e 3 linhas
    axes = axes.flatten()

    
    for i, year in enumerate(years):
        ax = axes[i]
        file_path = os.path.join(input_dir, str(year), f"Histograma_{year}_{mapa}.png")
        
        if os.path.exists(file_path):
            ax.imshow(Image.open(file_path))
            ax.set_title(f"{letra[i]}) {year}", fontsize=18)
        else:
            ax.set_title(f"{year} (Faltando)", fontsize=12, color='red')
        
        ax.axis('off')

    # Excluindo eixos extras caso o número de anos seja menor que 12
    for ax in axes[len(years):]:
        fig.delaxes(ax)

    # Título geral para o conjunto de mapas
    fig.suptitle(f"Comparação de Histogramas {mapa} (2014-2024)", fontsize=16)

    # Ajustando layout para evitar sobreposição
    fig.tight_layout(rect=[0, 0, 1, 0.95])

    # Salvando o gráfico para cada mapa separadamente
    plt.savefig(os.path.join(input_dir, f"Comparacao_Histogramas_{mapa}_2014_2024.png"), dpi=300, bbox_inches='tight')
    plt.show()





In [ ]:
###INGLES

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import rasterio
import os
import geopandas as gpd
import matplotlib.colors as mcolors
from rasterio.mask import mask
from shapely.geometry import mapping

# Definir base de dados para comparação - Cross Validation
df_results_subs = pd.read_hdf(general_path + '/Results_Erosivity_filled.h5', key=mapa)  # FILLED
df_results_subs = df_results_subs[df_results_subs['sum'] >= 8]  # Anos de dados
columns = [str(year) for year in range(2014, 2025)]
df_results_subs['mean'] = df_results_subs[columns].apply(lambda row: row[row != 0].mean(), axis=1)  

# Abrir imagem raster
output_raster_path = os.path.join(general_path, '2 - Result_images', f'{mapa}_mean_2014_2024.tif')  
with rasterio.open(output_raster_path) as src:  
    raster_crs = src.crs  # CRS do raster
    brasil_shapefile = brasil_shapefile.to_crs(raster_crs)  # Converter CRS do shapefile para o do raster
    
    # Recortar o raster usando o shapefile
    shapes = [mapping(geom) for geom in brasil_shapefile.geometry]  # Converter para formato compatível
    image, transform = mask(src.read(1)[::-1, :], shapes, crop=True)  # Aplicar máscara
    
    # Pegar apenas a primeira banda
    image = image[0]






# Ajustar os limites do raster recortado
minx, miny, maxx, maxy = brasil_shapefile.total_bounds  
extent = [minx, maxx, miny, maxy]  

colors = {
    0.0: "#FFFFFF",  # Branco (0)
    0.2: "#E3F2FD",  # Azul muito claro (2000)
    0.4: "#90CAF9",  # Azul claro (4000)
    0.6: "#42A5F5",  # Azul médio (6000)
    0.7: "#1565C0",  # Azul escuro (8000)
    0.85: "#B22222", # Vermelho médio (10000)
    1.0: "#8B0000",  # Vermelho escuro (12000+)
}

# Criar colormap suave
cmap = mcolors.LinearSegmentedColormap.from_list("custom_cmap", list(colors.items()))

# Criar normalização para os limites
norm = mcolors.Normalize(vmin=0, vmax=12000)

# Criar o mapa
fig, ax = plt.subplots(figsize=(9, 9))  
img_plot = ax.imshow(image, extent=extent, origin='lower', cmap=cmap, norm=norm)  
brasil_shapefile.plot(ax=ax, facecolor="none", edgecolor="black", linewidth=1)  
ax.grid(color='gray', linestyle='--', linewidth=0.5)  
ax.set_title("Mean of Kriging Interpolations", fontsize=14)  

# Adicionar barra de cores
cbar = plt.colorbar(img_plot, ax=ax, orientation='vertical', fraction=0.03, pad=0.04)  
cbar.ax.tick_params(labelsize=12)  
cbar.set_label(f" {unity[k]}", fontsize=14)  

plt.savefig(os.path.join(general_path, '2 - Result_images', 'presentation', f'INV_map_i_{mapa}.png'), dpi=300, bbox_inches='tight')  
plt.show()


In [ ]:
plt.rcParams.update({'font.family': 'Times New Roman'})


for k, mapa in enumerate(mapas):
    #DEFINIR BASE DE DADOS PARA COMPARAÇÃO - CROSS VALIDATION
    df_results_subs=pd.read_hdf(general_path +'/Results_Erosivity_filled.h5',key=mapa) # FILLED
    df_results_subs=df_results_subs[df_results_subs['sum']>=8] #anos de dados
    columns = [str(year) for year in range(2014, 2025)]

    df_results_subs['mean'] = df_results_subs[columns].apply(lambda row: row[row != 0].mean(), axis=1)  
    
    cmap = cmap_list[k]  
    # Open raster image  
    output_raster_path = os.path.join(general_path, '2 - Result_images', f'{mapa}_mean_2014_2024.tif')  
    with rasterio.open(output_raster_path) as src:  
        image = src.read(1)  
    bounds = brasil_shapefile.total_bounds  # [minx, miny, maxx, maxy]  
    minx, miny, maxx, maxy = bounds  
    extent = [minx, maxx, miny, maxy]  
    
    point_values = []  
    for _, row in df_results_subs.iterrows():  
        lon, lat = row['long'], row['lat']  
        y = math.ceil((lat + 33.75) * 150 / 39)  
        x = math.ceil((lon + 74) * 150 / 45.2)  
        point_values.append(image[y, x])  
    
    mae = mean_absolute_error(df_results_subs['mean'], point_values)  
    rmse = mean_squared_error(df_results_subs['mean'], point_values, squared=False)  
    r2 = r2_score(df_results_subs['mean'], point_values)  
    values = df_results_subs['mean']  
    
    # Creating the main map  
    fig, ax = plt.subplots(figsize=(9, 9))  
    ax.imshow(image, extent=extent, origin='lower', cmap=cmap)  
    brasil_shapefile.plot(ax=ax, facecolor="none", edgecolor="black", linewidth=1)  
    ax.grid(color='gray', linestyle='--', linewidth=0.5)  
    ax.set_title(f"Mean of Kriging Interpolations", fontsize=14)  
    norm = plt.Normalize(vmin=values.min(), vmax=values.max())  # Normalize  
    scatter = ax.scatter(df_results_subs['long'], df_results_subs['lat'], c=values, cmap=cmap, s=18,  
                         edgecolor='black', linewidth=0.7, label='Stations with 8 or more years of operation', alpha=0.75)  
    
    img_plot = ax.imshow(image, extent=extent, origin='lower', cmap=cmap)  
    cbar = plt.colorbar(img_plot, ax=ax, orientation='vertical', fraction=0.03, pad=0.04)  
    cbar.ax.tick_params(labelsize=12)  
    cbar.set_label(f" {unity[k]}", fontsize=14)  
    ax.legend(fontsize=11)  
    
    # Adding the text box in the lower right corner  
    stats_text = f"MAE: {mae:.2f}\nRMSE: {rmse:.2f}\nR²: {r2:.2f}"  
    ax.text(0.95, 0.05, stats_text, transform=ax.transAxes, fontsize=12, verticalalignment='bottom',  
            horizontalalignment='right', bbox=dict(facecolor='white', alpha=0.5))  
    
    # Adding the cross-validation plot in the lower left corner  
    from mpl_toolkits.axes_grid1.inset_locator import inset_axes  
    
    inset_ax = inset_axes(ax, width="27%", height="30%", loc='lower left', borderpad=5)  
    inset_ax.scatter(df_results_subs['mean'], point_values, s=10, color='darkblue', alpha=0.5)  
    inset_ax.plot([min(df_results_subs['mean']), max(df_results_subs['mean'])],  
                  [min(df_results_subs['mean']), max(df_results_subs['mean'])], color='red', linestyle='--')  
    inset_ax.set_title("Cross-validation", fontsize=12)  
    inset_ax.set_xlabel(f" {unity[k].split(' ')[0]} observed (Stations)", fontsize=10)  
    inset_ax.set_ylabel(f" {unity[k].split(' ')[0]} predicted (Raster)", fontsize=10)  
    inset_ax.tick_params(axis='both', which='major', labelsize=10)  
    inset_ax.grid(True, alpha=0.7)  
    inset_ax.patch.set_alpha(0.5)  # Transparent background for the plot  
    
    # Saving and displaying  
    plt.savefig(os.path.join(general_path, '2 - Result_images','presentation', f'map_EVALUATION_{mapa}.png'), dpi=300, bbox_inches='tight')  
    plt.show()


In [ ]:
fig, axes = plt.subplots(3, 3, figsize=(15, 12.5), constrained_layout=True)
for i, mapa in enumerate(mapas):
    output_std_dev_path = os.path.join(general_path, '2 - Result_images', f'{mapa}_std_dev_2014_2024.tif')
    output_mean_path = os.path.join(general_path, '2 - Result_images', f'{mapa}_mean_2014_2024.tif')
    
    with rasterio.open(output_mean_path) as src_mean, rasterio.open(output_std_dev_path) as src_std:
        image_mean = src_mean.read(1)
        image_std = src_std.read(1)
        image_c_v = image_std / image_mean
    bounds = brasil_shapefile.total_bounds  
    minx, miny, maxx, maxy = bounds  
    extent = [minx, maxx, miny, maxy]  
    
    # Assigning subplots for each row  
    ax_mean, ax_std, ax_cv = axes.flatten()[i * 3:i * 3 + 3]  
    
    # Mean map  
    img_plot_mean = ax_mean.imshow(image_mean, extent=extent, origin='lower', cmap=cmap_list[i])  
    brasil_shapefile.plot(ax=ax_mean, facecolor="none", edgecolor="black", linewidth=1)  
    ax_mean.set_title(f'{letra[i * 3]}) Mean - {unity[i]}', fontsize=16)  
    ax_mean.grid(color='gray', linestyle='--', linewidth=0.5)  
    cbar_mean = plt.colorbar(img_plot_mean, ax=ax_mean, orientation='vertical', fraction=0.03, pad=0.04)  
    cbar_mean.ax.tick_params(labelsize=14)  
    cbar_mean.set_label(f'{unity[i]}', fontsize=16)  
    
    # Standard deviation map  
    img_plot_std = ax_std.imshow(image_std, extent=extent, origin='lower', cmap='magma')  
    brasil_shapefile.plot(ax=ax_std, facecolor="none", edgecolor="black", linewidth=1)  
    ax_std.set_title(f'{letra[i * 3 + 1]}) Standard Deviation - {unity[i].split("(")[0]}', fontsize=16)  
    ax_std.grid(color='gray', linestyle='--', linewidth=0.5)  
    cbar_std = plt.colorbar(img_plot_std, ax=ax_std, orientation='vertical', fraction=0.03, pad=0.04)  
    cbar_std.ax.tick_params(labelsize=14)  
    cbar_std.set_label('Standard Deviation', fontsize=16)  
    
    # Coefficient of variation map  
    img_plot_cv = ax_cv.imshow(image_c_v, extent=extent, origin='lower', cmap='plasma', vmin=0, vmax=0.7)  
    brasil_shapefile.plot(ax=ax_cv, facecolor="none", edgecolor="black", linewidth=1)  
    ax_cv.set_title(f'{letra[i * 3 + 2]}) Coefficient of Variation - {unity[i].split("(")[0]} ', fontsize=16)  
    ax_cv.grid(color='gray', linestyle='--', linewidth=0.5)  
    cbar_cv = plt.colorbar(img_plot_cv, ax=ax_cv, orientation='vertical', fraction=0.03, pad=0.04)  
    cbar_cv.ax.tick_params(labelsize=14)  
    cbar_cv.set_label('Coeff. of Variation', fontsize=16)  

# Salvar e exibir o gráfico
plt.savefig(os.path.join(general_path, '2 - Result_images','presentation', 'map_Media_Desvio.png'), dpi=300, bbox_inches='tight')
plt.show()


In [ ]:

# Extract the necessary columns
latitude = df_data['lat'].values
longitude = df_data['long'].values
aqi_value = df_data[str(year)].values
# Define the grid for interpolation
bounds = brasil_shapefile.total_bounds  # [minx, miny, maxx, maxy]
minx, miny, maxx, maxy = bounds
gridx = np.linspace(minx , maxx , 150)
gridy = np.linspace(miny, maxy , 150)

## Ordinary Kriging

In [ ]:


# Perform Ordinary Kriging using the spherical variogram model
OK = OrdinaryKriging(longitude, 
                     latitude, 
                     aqi_value, 
                     variogram_model='spherical', 
                     coordinates_type='geographic', 
                     verbose=True, 
                     enable_plotting=True)
z_interp, ss = OK.execute('grid', gridx, gridy)

## Create overlay image of kriging interpolated values

In [ ]:
year = 2014
df_data = pd.read_hdf(f"{general_path}/Results_Erosivity_zero.h5", key='EI_30')
df_data=df_data[['long','lat',str(year),'state','gauge_code']].replace(0, np.nan).dropna().reset_index(drop=True)
df_data
latitude = df_data['lat'].values
longitude = df_data['long'].values
aqi_value = df_data[str(year)].values

# Perform Ordinary Kriging using the spherical variogram model
OK = OrdinaryKriging(longitude, 
                     latitude, 
                     aqi_value, 
                     variogram_model='spherical', 
                     coordinates_type='geographic', 
                     verbose=True, 
                     enable_plotting=True)
z_interp, ss = OK.execute('grid', gridx, gridy)

# Create a figure and axis
fig, ax = plt.subplots(figsize=(8, 8))

# Plot the interpolation results
cax = ax.imshow(z_interp, 
                extent=[gridx.min(), 
                        gridx.max(), 
                        gridy.min(), 
                        gridy.max()], 
                origin='lower', 
                cmap='viridis', 
                alpha=1)
ax.axis('off')

# Save the image
fig.savefig('kriging_interpolation.png', bbox_inches='tight', pad_inches=0, transparent=True)
plt.close(fig)

# Load the image with PIL
image = Image.open('kriging_interpolation.png')
fig          

## Create base map with overlayed interpolated data

In [ ]:



# Abrir a imagem interpolada recortada
image_path = "kriging_interpolation.png"
image = Image.open(image_path)

# Configurar os limites do recorte na escala geográfica
bounds = brasil_shapefile.total_bounds  # [minx, miny, maxx, maxy]
minx, miny, maxx, maxy = bounds
extent = [minx, maxx, miny, maxy]  # Extensão da área de interesse

# Criar a figura e os eixos
fig, ax = plt.subplots(figsize=(10, 10))

# Exibir a imagem interpolada como plano de fundo
ax.imshow(image, extent=extent, origin='upper', cmap='viridis')

# Plotar o shapefile do Brasil
brasil_shapefile.plot(ax=ax, facecolor="none", edgecolor="black", linewidth=1)

# Plotar os pontos utilizados
ax.scatter(longitude, latitude, color="blue", label="Pontos utilizados", zorder=3,marker='o',s=12,edgecolor="black",linewidth=0.35 )


# Configurar título e legenda
ax.set_title(f"Interpolação Kriging ordinário - {}", fontsize=14)
ax.legend()

# Salvar a figura
plt.savefig(f"Kriging_map_{year}.png", dpi=300, bbox_inches='tight')

# Mostrar a figura
plt.show()

In [ ]:
import rasterio
from rasterio.mask import mask
import geopandas as gpd
from PIL import Image

# Salvar a interpolação como um GeoTIFF
with rasterio.open(
    "interpolation.tif",
    "w",
    driver="GTiff",
    height=z_interp.shape[0],
    width=z_interp.shape[1],
    count=1,
    dtype=z_interp.dtype,
    crs="EPSG:4326",  # Certifique-se de usar a projeção correta
    transform=rasterio.transform.from_bounds(minx, miny, maxx, maxy, z_interp.shape[1], z_interp.shape[0]),
) as dst:
    dst.write(z_interp, 1)


# Recortar o GeoTIFF com base no shapefile
with rasterio.open("interpolation.tif") as src:
    out_image, out_transform = mask(src, brasil_shapefile.geometry, crop=True)
    out_meta = src.meta

# Atualizar metadados para o novo arquivo recortado
out_meta.update(
    {"driver": "GTiff", "height": out_image.shape[1], "width": out_image.shape[2], "transform": out_transform}
)

# Salvar o GeoTIFF recortado
with rasterio.open("interpolation_cropped.tif", "w", **out_meta) as dest:
    dest.write(out_image)

# Reabrir e plotar a imagem recortada
with rasterio.open("interpolation_cropped.tif") as src:
    image_cropped = src.read(1)
    extent = [src.bounds.left, src.bounds.right, src.bounds.bottom, src.bounds.top]

fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(image_cropped, extent=extent, origin="upper", cmap="magma")
brasil_shapefile.plot(ax=ax, facecolor="none", edgecolor="black", linewidth=1)
ax.scatter(longitude, latitude, color="blue", label="Pontos utilizados", zorder=3, marker="o")
ax.set_title("Interpolação Kriging Recortada", fontsize=14)
ax.legend()
plt.show()


In [ ]:
import folium
from folium.raster_layers import ImageOverlay

# Create a base map centered on Houston
m = folium.Map(location=[29.76, -95.37], zoom_start=10)

# Define the bounds where the image will be placed
bounds = [[gridy.min(), gridx.min()], [gridy.max(), gridx.max()]]

# Add the image overlay
image_overlay = ImageOverlay(
    image='kriging_interpolation.png',
    bounds=bounds,
    opacity=.7,
    interactive=True,
    cross_origin=False,
    zindex=1,
)

image_overlay.add_to(m)

# Add points for the measuring stations
for lat, lon in zip(latitude, longitude):
    folium.CircleMarker(
        location=[lat, lon],
        radius=2,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=1
    ).add_to(m)

# Save the map to an HTML file
m.save('houston_kriging_map_with_mesh.html')

# Display the map
m
